# FIT5202 Data processing for Big data

##  Activity: Parallel Search

For this tutorial, we will focus on parallel search in Big Data. Thus, the following sections will be done:
1. Review Data partitioning strategies
2. Implement distinct searching functionalities using RDDs: 
3. Implement distinct searching functionalities using Spark SQL module: you will use the Spark API to use dataframes and Spark SQL to perform the search functionality as in section 1.


Also, you will need to  visualise the parallelism on searching in these APIs and RDD implementation. Furthermore, you will need to look at the Query execution plan done by the Spark Optimizer Engine and understand how internally Spark executes or plans a searching function.

Let's get started.

## Table of Contents

* [SparkContext and SparkSession](#one)
* [Data Partitioning](#two)
* [Spark RDDs](#three)
    * [Data Partitioning in RDD](#three)
        * [Default Partitioning](#default)
        * [Hash Partitioning](#hash)
        * [Range Partitioning](#range)
    * [Parallel Search in RDDs](#parallel-search-rdd)    
* [Spark DataFrames](#dataframes)
    * [Data Partitioning in DataFrames](#df-partitioning)
    * [Parallel Search in DataFrames](#parallel_search_df)    
    * [Parallel Search with SparkSQL](#parallel_search_sparksql)       
* [Lab Tasks](#lab-task-1)
    * [Lab Task 1](#lab-task-1)
    * [Lab Task 2](#lab-task-2)
    * [Lab Task 3](#lab-task-3)
    * [Lab Task 4](#lab-task-4)


## SparkContext and SparkSession <a class="anchor" name="one"></a>

In [2]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Parallel Search"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

# # Method 2: Getting or instantiating a SparkContext
# sc = SparkContext.getOrCreate(spark_conf)
# sc.setLogLevel('ERROR')

sc

<SparkContext master=local[*] appName=Parallel Search>

## Data Partitioning <a class="anchor" id="two"></a>
In this first part of the tutorial, we will do a quick review of a few data partitioning strategies which we will need to know for the rest of the topics in this tutorial.

Data partitioning is the fundamental step for parallel search algorithms as parallelism in query and search processing is achieved through data partionining. 
In this activity, we will consider the following **three** partitioning strategies:
#### 1. Round-robin data partitioning ###
Round-robin data partitioning is the simplest data partitioning method in which each record in turn is allocated to a processing element (simply processor). Since it distributes the data evenly among all processors, it is also known as "equal-partitioning".

#### 2. Range data partitioning ###
Range data partitioning records based on a given range of the partitioning attribute. For example,the student table is partitioned based on "Last Name" based on the alphabetical order (i.e. A ~ Z). 

#### 3. Hash data partitioning ###
Hash data partitioning makes a partition based on a particular attribute using a hash function. The result of a hash function determines the processor where the record will be placed. Thus, all records within a partition have the same hash value.

## RDD partitioning <a class="anchor" id="three"></a>

By default, Spark partitions the data using <strong>Random equal partitioning</strong> unless there are specific transformations that uses a different type of partitioning</strong>
In the code below, we have defined two functions to implement custom partitioning using <strong>Range Partitioning</strong> and <strong>Hash Partitioning</strong>.


In [2]:
from pyspark.rdd import RDD

#A Function to print the data items in each RDD
#Same function is used to print out the data items of a DataFrame later
def print_partitions(data):
    if isinstance(data, RDD):
        numPartitions = data.getNumPartitions()
        partitions = data.glom().collect()
    else:
        numPartitions = data.rdd.getNumPartitions()
        partitions = data.rdd.glom().collect()
    
    print(f"####### NUMBER OF PARTITIONS: {numPartitions}")
    for index, partition in enumerate(partitions):
        # show partition if it is not empty
        if len(partition) > 0:
            print(f"Partition {index}: {len(partition)} records")
            # TODO: You can complete the code to show the values or some values for each partition
            print(partition)

In [3]:
#Sample data used for demonstrating the partitioning
list_tutors = [(1,'Aaditya'),(2,'Chinnavit'),(3,'Neha'),(4,'Huashun'),(5,'Mohammad'),
                (10,'Peter'),(11,'Paras'),(12, 'Tooba'),(3, 'David'),(18,'Cheng'),(9,'Haqqani')]

#Define the number of partitions
no_of_partitions = 4

In [4]:
(1,'Aaditya'),(2,'Chinnavit')
(3,'Neha'),(4,'Huashun')
(5,'Mohammad'),(10,'Peter')
(11,'Paras'),(12, 'Tooba')
(3, 'David'),(18,'Cheng')
(9,'Haqqani')

(9, 'Haqqani')

### Default Partitioning in Spark RDD <a class="anchor" id="default"></a>

In [6]:
# random equal partition
rdd = sc.parallelize(list_tutors, no_of_partitions)
rdd.collect()

[(1, 'Aaditya'),
 (2, 'Chinnavit'),
 (3, 'Neha'),
 (4, 'Huashun'),
 (5, 'Mohammad'),
 (10, 'Peter'),
 (11, 'Paras'),
 (12, 'Tooba'),
 (3, 'David'),
 (18, 'Cheng'),
 (9, 'Haqqani')]

In [6]:
print("Number of partitions:{}".format(rdd.getNumPartitions()))
print("Partitioner:{}".format(rdd.partitioner))
print_partitions(rdd)  

Number of partitions:4
Partitioner:None
####### NUMBER OF PARTITIONS: 4
Partition 0: 2 records
[(1, 'Aaditya'), (2, 'Chinnavit')]
Partition 1: 4 records
[(3, 'Neha'), (4, 'Huashun'), (5, 'Mohammad'), (10, 'Peter')]
Partition 2: 2 records
[(11, 'Paras'), (12, 'Tooba')]
Partition 3: 3 records
[(3, 'David'), (18, 'Cheng'), (9, 'Haqqani')]


<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#006DAE">TODO: </strong>How do you think the data is divided across the partitions by default when no partitoner is specified?</div>

### Hash Partitioning in RDD <a class="anchor" id="hash"></a>
Hash partitioning uses the formula <code>partition = hash_function() % numPartitions</code> to determine which partition data item falls into.

In [7]:
#Hash Function to implement Hash Partitioning 
#Just computes the sum of digits
#Example : hash_function(12) produces 3 i.e. 2 + 1
def hash_function(key):
    total = 0
    for digit in str(key):
        total += int(digit)
    return total

In [8]:
# hash partitioning
hash_partitioned_rdd = rdd.partitionBy(no_of_partitions, hash_function)
print_partitions(hash_partitioned_rdd)            

####### NUMBER OF PARTITIONS: 4
Partition 0: 1 records
[(4, 'Huashun')]
Partition 1: 5 records
[(1, 'Aaditya'), (5, 'Mohammad'), (10, 'Peter'), (18, 'Cheng'), (9, 'Haqqani')]
Partition 2: 2 records
[(2, 'Chinnavit'), (11, 'Paras')]
Partition 3: 3 records
[(3, 'Neha'), (12, 'Tooba'), (3, 'David')]


<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#006DAE">Note: </strong>Look at how the data is partitioned. For example, Partition 0 has 1 record, [(4, 'Huashun')]. Here is the step-wise breakdown:
    <ul>
        <li>hash_function(4) = 4</li>
        <li>Partition for the key of 4 is determined by <code>hash_function(4)%numPartitions</code> i.e. 4%4=0</li>
        <li>Similarly, for (18,'Cheng'), partition is given by <code>hash+function(18)%numPartitions</code> i.e. 9%4=1</li>
    </ul>

</div>

### Range Partitioning in RDD <a class="anchor" id="range"></a>
This strategy uses a range to distribute the items to respective partitions when the keys fall within the range. 

In [3]:
no_of_partitions=4

#Find the size of the elements in RDD
chunk_size = len(list_tutors)/no_of_partitions
#Define a range of values by key to distribute across partitions
#Here for simplicity, we are defining the range i.e. keys from 1-4 to fall in first partition, 5-9 in second partition and so on
range_arr=[[1,4],[5,9],[10,14],[15,19]]

def range_function(key):
    for index,item in enumerate(range_arr):
        if key >=item[0] and key <=item[1]:
            return index


In [10]:
# range partition
range_partitioned_rdd = rdd.partitionBy(no_of_partitions, range_function)
print_partitions(range_partitioned_rdd)

####### NUMBER OF PARTITIONS: 4
Partition 0: 5 records
[(1, 'Aaditya'), (2, 'Chinnavit'), (3, 'Neha'), (4, 'Huashun'), (3, 'David')]
Partition 1: 2 records
[(5, 'Mohammad'), (9, 'Haqqani')]
Partition 2: 3 records
[(10, 'Peter'), (11, 'Paras'), (12, 'Tooba')]
Partition 3: 1 records
[(18, 'Cheng')]


## Parallel Search using RDDs  <a class="anchor" id="parallel-search-rdd"></a>

Now we will implement basic search functionalities and visualise the parallelism embedded in Spark to perform these kind of queries.

In this tutorial, you will use a csv dataset **bank.csv**. However, for this tutorial we won't analyse the case study but only perform some search queries with this data

In [11]:
# Using Spark, we can read and load a csv file
# Read csv file and load into an RDD object
bank_rdd = sc.textFile('bank.csv')

# If you want to specify the number of partitions, you can add the number as a second argument
# bank_rdd = sc.textFile('bank.csv', 10)

## Exploring the data file, we can see that it contains different types of information
## Some useful information is printed below
print(f"Total partitions: {bank_rdd.getNumPartitions()}")
print(f"Number of lines: {bank_rdd.count()}")

## Each element of the RDD is a line from the file
bank_rdd.take(4)

Total partitions: 2
Number of lines: 11163


['age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit',
 '59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes',
 '56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes',
 '41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes']

### Search in RDDs based on multiple conditions

We will focus on only four attributes from the data: age, education, marital and balance for filtering conditions. However, we will display additional information as well.

In [12]:
# 1. Split each line separated by comma into a list 
bank_rdd1 = bank_rdd.map(lambda line: line.split(','))
# 2. Remove the header
header = bank_rdd1.first()
bank_rdd1 = bank_rdd1.filter(lambda row: row != header)   #filter out header

# Indices for each attribute we will use
# Filter: age, education, marital, balance = 0, 3, 2, 5
# Display additional: day, month, deposit = 9, 10, 16

# 3. Search the records with balance between 1000 and 2000
bank_rdd1 = bank_rdd1.filter(lambda x: int(x[5])>1000 and int(x[5])<2000)
# 4. Also search the records with primrary or secondary education and age less than 30
bank_rdd1 = bank_rdd1.filter(lambda x: x[3] in ['primary','secondary'] and int(x[0])<30)
# 5. Also filter with those who are married
bank_rdd1 = bank_rdd1.filter(lambda x: x[2]=='married' )
# 6. Display the previous attributes plus the information of day, month and deposit
bank_rdd1 = bank_rdd1.map(lambda field: (field[0],field[2],field[3],field[5],
                                         field[9],field[10],field[16]))
# Print how many final records
print(bank_rdd1.count())

18


In [13]:
# Let's see how the data was divided and the data for each partition
numPartitions = bank_rdd1.getNumPartitions()
print(f"Total partitions: {numPartitions}")

# glom(): Return an RDD created by coalescing all elements within each partition into a list
partitions = bank_rdd1.glom().collect()
for index,partition in enumerate(partitions):
    print(f'------ Partition {index}:')
    for record in partition:
        print(record)

Total partitions: 2
------ Partition 0:
('29', 'married', 'secondary', '1135', '17', 'feb', 'yes')
('27', 'married', 'secondary', '1293', '8', 'apr', 'yes')
('29', 'married', 'secondary', '1180', '17', 'apr', 'yes')
('28', 'married', 'secondary', '1086', '20', 'apr', 'yes')
('26', 'married', 'secondary', '1595', '15', 'jun', 'yes')
('27', 'married', 'secondary', '1596', '1', 'sep', 'yes')
('28', 'married', 'secondary', '1595', '9', 'sep', 'yes')
('27', 'married', 'secondary', '1595', '29', 'dec', 'yes')
------ Partition 1:
('26', 'married', 'secondary', '1417', '6', 'jun', 'no')
('23', 'married', 'secondary', '1309', '3', 'jun', 'no')
('24', 'married', 'secondary', '1222', '20', 'apr', 'no')
('28', 'married', 'secondary', '1238', '14', 'may', 'no')
('26', 'married', 'secondary', '1595', '2', 'mar', 'no')
('27', 'married', 'secondary', '1303', '21', 'may', 'no')
('25', 'married', 'secondary', '1782', '19', 'jun', 'no')
('28', 'married', 'secondary', '1137', '6', 'feb', 'no')
('28', 'mar

<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#006DAE">TODO: </strong>Verify the parallelism in the Spark UI and explore the content. How many jobs have been executed so far?</div>

### Searching max/min value of an attribute in an RDD
This task will aim to find the record in the dataset that contains the highest value for a given attribute. In this case the attribute chosen is "balance".

In [14]:
# Read csv but now with 4 partitions
bank_rdd_4 = sc.textFile('bank.csv',4)

# Split and remove the header
bank_rdd_4 = bank_rdd_4.map(lambda line: line.split(','))
header = bank_rdd_4.first()
bank_rdd_4 = bank_rdd_4.filter(lambda row: row != header)   #filter out header

# Display the first 3 records
bank_rdd_4.take(3)

[['59',
  'admin.',
  'married',
  'secondary',
  'no',
  '2343',
  'yes',
  'no',
  'unknown',
  '5',
  'may',
  '1042',
  '1',
  '-1',
  '0',
  'unknown',
  'yes'],
 ['56',
  'admin.',
  'married',
  'secondary',
  'no',
  '45',
  'no',
  'no',
  'unknown',
  '5',
  'may',
  '1467',
  '1',
  '-1',
  '0',
  'unknown',
  'yes'],
 ['41',
  'technician',
  'married',
  'secondary',
  'no',
  '1270',
  'yes',
  'no',
  'unknown',
  '5',
  'may',
  '1389',
  '1',
  '-1',
  '0',
  'unknown',
  'yes']]

In [15]:
# Using the RDD function max(), it can be obtained in a single line 
result_max_balance = bank_rdd_4.max(key=lambda x: x[5]) # Get max by value in index 5 (balance)
# Print the record obtain with highest balance
print(result_max_balance)

['58', 'self-employed', 'married', 'secondary', 'no', '9994', 'no', 'no', 'cellular', '10', 'jul', '400', '1', '-1', '0', 'unknown', 'no']


In [16]:
# Get record with balance 10576
bank_record = bank_rdd_4.filter(lambda x: x[5]=='10576').collect()
print(bank_record)

[['29', 'management', 'married', 'tertiary', 'no', '10576', 'no', 'no', 'unknown', '15', 'may', '1224', '2', '-1', '0', 'unknown', 'yes']]


<a class="anchor" id="lab-task-1"></a>
<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#FF5555">1. Lab Task: </strong>Compare the <code>result_max_balance</code> record with the record above <code>(bank_record)</code>. Was the record obtained previously correct i.e. <code>result_max_balance</code>? <b>Explain what happened.</b></div>


In [17]:
result_max_balance = bank_rdd_4.max(key=lambda x: x[5])

<a class="anchor" id="lab-task-2"></a>
<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#FF5555">2. Lab Task: </strong>As you noticed in the previous result, the record returned originally (result_max_balance) was incorrect.
    <p><i>Fix the code below that uses the <code>max()</code> function to get the record with the correct maximum balance.</p></i></div>


#### To learn more about functions in RDDs, you can look into the next 2 sites:
1. http://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations
1. https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_Cheat_Sheet_Python.pdf

## DataFrames in Spark <a class="anchor" id="dataframes"></a>
A DataFrame is a distributed collection of data organized into named columns. It is equivalent to a table in relational database or a dataframe in R/Python but with richer optimizations under the hood. For more information visit : 

https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html

### Creating DataFrames
SparkSession provides an easy method <code>createDataFrame</code> to create Spark DataFrames. Data can be loaded from csv, json, xml and other sources like local file system or HDFS. More information on : 
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

To display the schema, i.e. the  structure of the DataFrame, you can use <strong>printSchema()</strong> method.

In [18]:
df = spark.createDataFrame([(1,'Aaditya','A'),(2,'Chinnavit','C'),(3,'Neha','N'),(4,'Huashun','H'),(5,'Mohammad','M'),
                            (10,'Prajwol', 'P'),(1,'Paras','P'),(1, 'Tooba','T'),(3, 'David','D'),(4,'Cheng','C'),(9,'Haqqani','H')],
                           ['Id','Name','Initial'])

#display the rows of the dataframe
df.show(5)
#view the schema
df.printSchema()

+---+---------+-------+
| Id|     Name|Initial|
+---+---------+-------+
|  1|  Aaditya|      A|
|  2|Chinnavit|      C|
|  3|     Neha|      N|
|  4|  Huashun|      H|
|  5| Mohammad|      M|
+---+---------+-------+
only showing top 5 rows

root
 |-- Id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Initial: string (nullable = true)



Another way to create a DataFrame is use the <strong>spark.read.csv</strong> file to load the data from csv to a DataFrame

In [19]:
df = spark.read.csv("bank.csv",header=True)

<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#006DAE">TODO: </strong>Display first 10 rows of the above dataframe. 
Try out other dataframe methods:    
    <ul>
        <li><code>df.columns, df.count()</code></li>
        <li><code>df.describe('column_name').show()</code></li>
        <li><strong>Selecting:</strong><code>df.select('column_name').show(), df.select('column_name').distinct().show()</code>
        </li>
        <li><strong>Filtering:</strong><code>df.filter(df.column_name == 123).show()</code></li>
</div>

### Partitioning in DataFrames <a class="anchor" id="df-partitioning"></a>

In [20]:
df = spark.createDataFrame([(1,'Aaditya','A'),(2,'Chinnavit','C'),(3,'Neha','N'),(4,'Huashun','H'),(5,'Mohammad','M'),
                            (10,'Prajwol', 'P'),(1,'Paras','P'),(1, 'Tooba','T'),(3, 'David','D'),(4,'Cheng','C'),(9,'Haqqani','H')],
                           ['Id','Name','Initial'])

In [21]:
# Round-robin data partitioning
df_round = df.repartition(2)
# Range data partitioning
df_range = df.repartitionByRange(2, "Initial")
# Hash data partitioning
column_hash = "Id"
df_hash = df.repartition(column_hash)

In [22]:
print_partitions(df_round)

####### NUMBER OF PARTITIONS: 2
Partition 0: 6 records
[Row(Id=2, Name='Chinnavit', Initial='C'), Row(Id=3, Name='Neha', Initial='N'), Row(Id=5, Name='Mohammad', Initial='M'), Row(Id=10, Name='Prajwol', Initial='P'), Row(Id=9, Name='Haqqani', Initial='H'), Row(Id=4, Name='Cheng', Initial='C')]
Partition 1: 5 records
[Row(Id=1, Name='Aaditya', Initial='A'), Row(Id=4, Name='Huashun', Initial='H'), Row(Id=1, Name='Tooba', Initial='T'), Row(Id=1, Name='Paras', Initial='P'), Row(Id=3, Name='David', Initial='D')]


In [23]:
print_partitions(df_range)

####### NUMBER OF PARTITIONS: 2
Partition 0: 6 records
[Row(Id=1, Name='Aaditya', Initial='A'), Row(Id=2, Name='Chinnavit', Initial='C'), Row(Id=4, Name='Huashun', Initial='H'), Row(Id=3, Name='David', Initial='D'), Row(Id=4, Name='Cheng', Initial='C'), Row(Id=9, Name='Haqqani', Initial='H')]
Partition 1: 5 records
[Row(Id=3, Name='Neha', Initial='N'), Row(Id=5, Name='Mohammad', Initial='M'), Row(Id=10, Name='Prajwol', Initial='P'), Row(Id=1, Name='Paras', Initial='P'), Row(Id=1, Name='Tooba', Initial='T')]


In [24]:
print_partitions(df_hash)

####### NUMBER OF PARTITIONS: 200
Partition 49: 1 records
[Row(Id=9, Name='Haqqani', Initial='H')]
Partition 68: 1 records
[Row(Id=5, Name='Mohammad', Initial='M')]
Partition 69: 3 records
[Row(Id=1, Name='Aaditya', Initial='A'), Row(Id=1, Name='Paras', Initial='P'), Row(Id=1, Name='Tooba', Initial='T')]
Partition 77: 1 records
[Row(Id=10, Name='Prajwol', Initial='P')]
Partition 107: 2 records
[Row(Id=3, Name='Neha', Initial='N'), Row(Id=3, Name='David', Initial='D')]
Partition 128: 1 records
[Row(Id=2, Name='Chinnavit', Initial='C')]
Partition 140: 2 records
[Row(Id=4, Name='Huashun', Initial='H'), Row(Id=4, Name='Cheng', Initial='C')]


In [25]:
# Read csv file and load into a dataframe
df = spark.read.csv("bank.csv",header=True)

<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#006DAE">TODO: </strong> How many partitions the dataframe have?</div>

<a class="anchor" id="lab-task-3"></a>
<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:##FF5555">3. Lab Task: </strong> Implement Range and Hash Partitioning techniques for the new dataset and display the partitions. COMPLETE THE CODE BELOW.</div>

In [26]:
## We can specify how many partitions or what kind of partitioning we want for a dataframe
# Round-robin data partitioning
df_round = df.repartition(5)
# Range data partitioning
df_range = df.repartitionByRange(5,"balance")
# Hash data partitioning
column_hash = "education"
df_hash = df.repartition(column_hash) 

In [27]:
## Print the number of partitions for each dataframe
print(f"----- NUMBER OF PARTITIONS df_round: {df_round.rdd.getNumPartitions()}")
print(f"----- NUMBER OF PARTITIONS df_range: {df_range.rdd.getNumPartitions()}")
print(f"----- NUMBER OF PARTITIONS df_hash: {df_hash.rdd.getNumPartitions()}")

## Verifying the number of partitions for the dataframe with hash partitioning it would indicate 200.
## One important thing is that by default, when the number of partitions are not indicated,
## The default number of partitions is 200

## However, most of the partitions for df_hash are empty.
## We can verify by using the following function

## df is the dataframe to show its information
def print_partitions(df):
    numPartitions = df.rdd.getNumPartitions()
    print(f"####### NUMBER OF PARTITIONS: {numPartitions}")
    partitions = df.rdd.glom().collect()
    for index,partition in enumerate(partitions):
        # show partition if it is not empty
        if len(partition)>0:
            print(f"Partition {index}: {len(partition)} records")
            # TODO: You can complete the code to show the values or some values for each partition
            

----- NUMBER OF PARTITIONS df_round: 5
----- NUMBER OF PARTITIONS df_range: 5
----- NUMBER OF PARTITIONS df_hash: 200


<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#006DAE">TODO: </strong> Complete the above code to show the values for each partition</div>

In [28]:
## You can verify the partitioning and the query plan when an action is performed with the function explain()
# Query plan for df_round
df_round.explain()
# Query plan for df_range
df_range.explain()
# Query plan for df_hash
df_hash.explain()

== Physical Plan ==
Exchange RoundRobinPartitioning(5), false, [id=#83]
+- FileScan csv [age#91,job#92,marital#93,education#94,default#95,balance#96,housing#97,loan#98,contact#99,day#100,month#101,duration#102,campaign#103,pdays#104,previous#105,poutcome#106,deposit#107] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/student/5202/session2/bank.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:string,job:string,marital:string,education:string,default:string,balance:string,housin...


== Physical Plan ==
Exchange rangepartitioning(balance#96 ASC NULLS FIRST, 5), false, [id=#90]
+- FileScan csv [age#91,job#92,marital#93,education#94,default#95,balance#96,housing#97,loan#98,contact#99,day#100,month#101,duration#102,campaign#103,pdays#104,previous#105,poutcome#106,deposit#107] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/student/5202/session2/bank.csv], PartitionFilters: [], PushedFilters: [], 

### Parallel Search using Spark Dataframe <a class="anchor" id="parallel_search_df"></a>

We will perform the same filtering criteria as in section 1. This time the logic won't be implemented by us but just declare by using the functions of the Spark Dataframe API to perform the same queries. Thus, we should obtain the same results as before.

Furthermore, now you will need to see in the Spark UI, the RDD DAG Visualisation and the Execution Plan of the queries performed with the function explain() as we did previously.

<a class="anchor" id="lab-task-4"></a>
<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:##FF5555">4. Lab Task: </strong> Complete the code in the given cell below to implement the given conditions.</div>

In [29]:
# Using the Spark Dataframe API we can obtain the dataframe for a csv file
# We already created dataframes with different types of partitioning
# Choose one of them to work with and perform the queries made in section 1
from pyspark.sql.functions import col

bank_df = df_round

## The functions to filter in dataframes are similar to the functions in RDD. Thus, the steps are:
# 1. Search the records with balance between 1000 and 2000
bank_df = bank_df.filter(col("balance")>1000)\
            .filter(col("balance")<2000)
# TODO: 
# 2. Also in the same dataframe, search the records with primary or secondary education and age less than 30
bank_df =

# TODO:
# 3. Also filter with those who are married
bank_df = 


# TODO:
# 4. Display the previous attributes plus the information of day, month and deposit
bank_df = 


# 5. Display the records
bank_df.show()

SyntaxError: invalid syntax (<ipython-input-29-e12408b9fd18>, line 14)

In [ ]:
#### Query and partition information
print_partitions(bank_df)
#### Execution Plan for query with multiple filter conditions
bank_df.explain()

<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#006DAE">TODO: </strong> Repeat the same query with different partitioning strategies <strong>(Round-Robin, Range and Hash)</strong> and compare its query execution plan plus its information in the <strong>Spark UI.</strong> </div>

<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:##333333">EXAMPLE: </strong>Obtain also the <code>max/min</code> as you did in RDDs but now using <strong>Spark DataFrame</strong>. Does it return the same value as in section 1? Also, check its execution plan and the information in Spark UI.</div>

In [ ]:
#### SOLUTION:
bank_max_balance = df_round.selectExpr("int(balance)").selectExpr("max(balance)").collect()
print(bank_max_balance)

### Parallel Search using SQL language in Spark  <a class="anchor" id="parallel_search_sparksql"></a>
#### Spark SQL
Spark SQL is a Spark module for structured data processing. It provides a programming abstraction called DataFrames and can also act as a distributed SQL query engine. It enables unmodified Hadoop Hive queries to run up to 100x faster on existing deployments and data. It also provides powerful integration with the rest of the Spark ecosystem (e.g., integrating SQL query processing with machine learning). <a href="https://databricks.com/glossary/what-is-spark-sql#:~:text=Spark%20SQL%20is%20a%20Spark,on%20existing%20deployments%20and%20data">[Read More]</a>.

A view can be created from a dataframe in order to use SQL queries to search data. In this section, you will use SQL statements to perform search queries in the views that will be registered from the dataframes we created in the previous section.

In [ ]:
# register the original DataFrame as a temp view so that we can query it using SQL
df.createOrReplaceTempView("df_sql")
filter_sql = spark.sql('''
  SELECT age,education,balance,day,month,deposit
  FROM df_sql
  WHERE balance between 1000 and 2000
  AND education in ('secondary','primary')
  AND age < 30
  AND marital = 'married'
''')
# filter_sql.explain()
filter_sql.collect()

<a class="anchor" id="lab-task-6"></a>
<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#333333">EXAMPLE: </strong>Obtain also the <code>max/min</code> as you did in RDDs and DataFrames, but now using <strong>Spark SQL</strong>. Does it return the same value as in previous cases? Also, check its execution plan and the information in Spark UI.</div>

In [ ]:
#### SOLUTION:
max_sql = spark.sql('''
  SELECT MAX(CAST(balance AS INT)) as max_balance
  FROM df_sql
''')
# Check the result obtained
max_sql.collect()

**Congratulations on finishing this activity!**

Having practiced today's activities, we're now ready to embark on a trip of the rest of exiciting FIT5202 activities! See you next week!